# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

import pickle

### 2. Load the dataset

In [3]:
# loading the dataset
df = pd.read_csv('car+data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [ ]:
print('shape: ',df.shape)
print('Basic information:' )
df.info()

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [5]:
# checking for duplicated records in the dataset

print('Number of duplicated records :',df.duplicated().sum())

The Number of duplicated records in the dataset : 2


In [6]:
# the duplicated records
df[df.duplicated()]

The duplicated records : 


,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
17,ertiga,2016,7.75,10.79,43000,Diesel,Dealer,Manual,0
93,fortuner,2015,23.00,30.61,40000,Diesel,Dealer,Automatic,0


In [7]:
# getting the duplicated records
duplicate_records = df[df.duplicated()]

if not duplicate_records.empty:
    print("Duplicate records found. Dropping duplicates...")
    # Dropping duplicate records
    df = df.drop_duplicates()
    print("Duplicates dropped. Updated shape of the data:", df.shape)
else:
    print("No duplicate records found.")

Duplicate records found. Dropping duplicates...
Duplicates dropped. Updated shape of the data: (299, 9)


### 5. Drop the columns which you think redundant for the analysis.

In [ ]:
# the Car_Name column is not required for further analysis.
drop_cols = ['Car_Name']

df = df.drop(columns = drop_cols)

# checking if the columns are dropped or not
all_removed = True
for col_name in drop_cols:
    if col_name in df.columns:
        all_removed = False
        break
        
if all_removed:
    print("The column(s) have been removed.")
else:
    print("The column(s) have not been removed.")

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [9]:
# computing age of the car = current year - year of purchase
# creating new column as required
df['age_of_the_car'] = 2024 - df['Year']

df.head()

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


In [10]:
# dropping 'year' feature
df.drop('Year', axis=1, inplace=True)

In [11]:
df.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


### 7. Encode the categorical columns

In [12]:
# get unique sub-categories within each category

print('Unique Categories in "Fuel_Type"',df['Fuel_Type'].unique())
print('Unique Categories in "Seller_Type"',df['Seller_Type'].unique())
print('Unique Categories in "Transmission"',df['Transmission'].unique())

Unique Categories in "Fuel_Type" ['Petrol' 'Diesel' 'CNG']
Unique Categories in "Seller_Type" ['Dealer' 'Individual']
Unique Categories in "Transmission" ['Manual' 'Automatic']


In [13]:
# Manual encoding for categorical variables

# Fuel_Type
fuel_type_map = {'Petrol': 0, 'Diesel': 1, 'CNG': 2}
df['Fuel_Type'] = df['Fuel_Type'].replace(fuel_type_map)
# Seller_Type
seller_type_map = {'Dealer': 0, 'Individual': 1}
df['Seller_Type'] = df['Seller_Type'].replace(seller_type_map)

# Transmission
transmission_map = {'Manual': 0, 'Automatic': 1}
df['Transmission'] = df['Transmission'].replace(transmission_map)

In [14]:
# checking the dataset after label encoding 
df.sample(5)

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
103,1.45,1.60,1200,0,1,0,0,7
214,5.25,5.70,20114,0,0,0,0,7
204,2.75,4.43,28282,0,0,0,0,9
57,4.75,18.54,50000,0,0,0,0,14
246,3.75,6.79,35000,0,0,0,0,12


### 8. Separate the target and independent features.

In [15]:
# Independant features
X = df.drop('Selling_Price', axis=1)

# target
y = df['Selling_Price']

In [16]:
print('Independant Features : ')
X.head()

Independant Features : 


,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,5.59,27000,0,0,0,0,10
1,9.54,43000,1,0,0,0,11
2,9.85,6900,0,0,0,0,7
3,4.15,5200,0,0,0,0,13
4,6.87,42450,1,0,0,0,10


In [17]:
print('Target Feature : ')
y.head()

Target Feature : 


0    3.35
1    4.75
2    7.25
3    2.85
4    4.60
Name: Selling_Price, dtype: float64

### 9. Split the data into train and test.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [19]:
# Build Random Forest Regressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on train and test sets
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Calculate R2 score for train and test sets
train_r2_score = r2_score(y_train, train_predictions)
test_r2_score = r2_score(y_test, test_predictions)

print("R2 score for train set:", train_r2_score)
print("R2 score for test set:", test_r2_score)

R2 score for train set: 0.9826728122912187
R2 score for test set: 0.9032720354187607


### 11. Create a pickle file with an extension as .pkl

In [23]:
# putting the model in a pickle file
pickle.dump(model, open('sp.pkl', 'wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Used Car Selling Price Prediction</title>
</head>

<body>

    <div class="hero-image">
      <div class="hero-text">

        <h1 style="font-size:50px">Used Car Price Predictor</h1>
         <br><br><h3>{{ prediction_text }}<h3>
      </div>
    </div>

     <style>

        body, html {
          height: 100%;
          margin: 0;
          font-family: Arial, Helvetica, sans-serif;
        }

        .hero-image {
          background-image: linear-gradient(rgba(0, 0, 0, 0.5), rgba(0, 0, 0, 0.5)), url('/static/bg_image.jpg');
          height: 50%;
          background-position: bottom;
          background-repeat: no-repeat;
          background-size: cover;
          position: relative;
        }

        .hero-text {
          text-align: center;
          position: absolute;
          top: 50%;
          left: 50%;
          transform: translate(-50%, -50%);
          color: white;
        }

    </style>


    <div style="color:	rgb(0, 0, 0)">
        <form action="{{ url_for('predict')}}" method="post">

            <h2>Enter Car Details: </h2>
            
            <h3>Age of the car(In years)</h3>
                <input id="first" name="car_age" type="number ">

            <h3>Present Showroom Price(In lakhs)</h3>
                <input id="second" name="current_price" required="required">
            
            <h3>Kilometers Driven</h3>
                <input id="third" name="kms" required="required">
            
            <h3>Owner Type (0/1/3)</h3>
                <input id="fourth" name="owner" required="required">
            
            <h3>Fuel type</h3>
            <select name="fuel_type" id="fuel" required="required">
                <option value="0">Petrol</option>
                <option value="1">Diesel</option>
                <option value="2">CNG</option>
            </select>
            
            <h3>Seller Type</h3>
            <select name="seller_type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
            
            <h3>Transmission type</h3><select name="transmission" id="research" required="required">
                <option value="0">Manual Car</option>
                <option value="1">Automatic Car</option>
            </select>
            
            <br><br>
                <button id="sub" type="submit ">Predict Selling Price</button>
            <br>

        </form>

    </div>

    <style>
	body {
            background-color: 101, 10, 20;
            text-align: center;
            padding: 0px;
	    font-family: Helvetica;
        }

        #research {
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }

        #box {
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }

        #fuel {
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }

        #fuel:hover {
            background-color: white;
        }

        #research {
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #research:hover {
            background-color: white;
        }

        #resea {
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #resea:hover {
            background-color: white;
        }

        #sub {
            background-color: Green;
            font-family:'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }

        #sub:hover {
            background-color: white;
        }

        #first {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #second {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #third {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #fourth {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>

</html>

### b) Create app.py file and write the predict function

### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

- <font color=red> app.py file is uploaded within the repository folder files.Trigger it from command line or VS Code </font>

### Happy Learning :)